# Importing Necessary libraries

In [4]:
import requests
import bs4 
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
url = 'https://www.dsebd.org/company_listing.php'
#Function definition for getting the page
def get_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    return soup

In [6]:
#calling the get-page function
soup = get_page(url)

In [7]:
#function definition for getting the company_url
def get_company_url(soup):
    Company_url = []
    base_url = 'https://www.dsebd.org/'
    links = soup.find_all('a', class_ ='abhead')
    for link in links:
        url = base_url + link['href']
        Company_url.append(url)
    return Company_url

In [8]:
#calling the get_company_url function
Company_url = get_company_url(soup)

In [9]:
Company_url[:10]

['https://www.dsebd.org/displayCompany.php?name=1JANATAMF',
 'https://www.dsebd.org/displayCompany.php?name=1STPRIMFMF',
 'https://www.dsebd.org/displayCompany.php?name=AAMRANET',
 'https://www.dsebd.org/displayCompany.php?name=AAMRATECH',
 'https://www.dsebd.org/displayCompany.php?name=ABB1STMF',
 'https://www.dsebd.org/displayCompany.php?name=ABBANK',
 'https://www.dsebd.org/displayCompany.php?name=ACFL',
 'https://www.dsebd.org/displayCompany.php?name=ACI',
 'https://www.dsebd.org/displayCompany.php?name=ACIFORMULA',
 'https://www.dsebd.org/displayCompany.php?name=ACMELAB']

In [10]:
len(Company_url)

384

# Finding the Columns

In [11]:
#function for getting the columns
def get_columns(random):
    url = random
    r = requests.get(url)
    soup1 = BeautifulSoup(r.content, 'lxml')
    
    
    Columns = ['Company_name', 'Trading_name']
    
    #Creating Columns for market information
    Market_Information_table = soup1.find('table', class_='table table-bordered background-white')
    for column in Market_Information_table.find_all('th'):
        Columns.append(column.text.strip())
    # insert New column as this was missing
    Columns.insert(8, 'Change Percentage') 

    #Basic information table is 1th index in the list
    #Creating Column for basic information
    Basic_Information_table = soup1.find_all('table', class_='table table-bordered background-white')[1]
    for column in Basic_Information_table.find_all('th')[:8]:
        Columns.append(column.text.strip())

    # Index of Company addres is 11
    #Creating Columns for the Company address
    address = []
    Company_address = soup1.find_all('table', class_='table table-bordered background-white', id = 'company')[11]
    for column in Company_address.find_all('td'):
        data = column.text.strip()
        address.append(data)
    for i in range(0, len(address), 2):
        Columns.append(address[i])    
        
    return Columns

In [12]:
#calling the column generating function foe any random company url
ran = Company_url[1]
Columns = get_columns(ran)
Columns

['Company_name',
 'Trading_name',
 'Last Trading Price',
 'Closing Price',
 'Last Update',
 "Day's Range",
 'Change*',
 "Day's Value (mn)",
 'Change Percentage',
 "52 Weeks' Moving Range",
 'Opening Price',
 "Day's Volume (Nos.)",
 'Adjusted Opening Price',
 "Day's Trade (Nos.)",
 "Yesterday's Closing Price",
 'Market Capitalization (mn)',
 'Authorized Capital (mn)',
 'Debut Trading Date',
 'Paid-up Capital (mn)',
 'Type of Instrument',
 'Face/par Value',
 'Market Lot',
 'Total No. of Outstanding Securities',
 'Sector',
 'Address',
 'Contact Phone',
 'Fax',
 'E-mail',
 'Web Address',
 'Company Secretary Name',
 'Cell No.',
 'Telephone No.',
 'E-mail']

# Finding the value under each columns

In [14]:
Company_data = []
for url in Company_url:
    
    response = requests.get(url)
    soup1 = BeautifulSoup(response.content, 'lxml')


    Data = []
    #Finding Company_name.
    Company_name = soup1.find('h2', class_ = 'BodyHead topBodyHead').text.split(':')[1]
    Company_name = Company_name.split('\n')[0].strip()
    Data.append(Company_name)

    #Splinting the Trading_name.
    Trading_name = url.split('=')[1]
    Data.append(Trading_name)

    # Getting the table data from market information table
    Market_Information_table = soup1.find('table', class_='table table-bordered background-white')
    for column_data in Market_Information_table.find_all('td'):
        Data.append(column_data.text.strip())
    
    # Getting the table data from basic information table
    Basic_Information_table = soup1.find_all('table', class_='table table-bordered background-white')[1]
    for column_data in Basic_Information_table.find_all('td')[:8]:
        Data.append(column_data.text.strip())

    # Getting the table data from company address table
    add_data = []
    Company_address = soup1.find_all('table', class_='table table-bordered background-white')[11]
    for column_data in Company_address.find_all('td'):
        data = column_data.text.strip()
        add_data.append(data)

    for i in range(1, len(add_data), 2):
        Data.append(add_data[i]) 

    Data_t = tuple(Data) # Converting the list into touple 
    Company_data.append(Data_t) # Appending touple into the list for the inserting porpuse in the sqlite data

In [16]:
len(Company_data)

384

In [17]:
Columns_t = tuple(Columns)
print(Columns_t)

('Company_name', 'Trading_name', 'Last Trading Price', 'Closing Price', 'Last Update', "Day's Range", 'Change*', "Day's Value (mn)", 'Change Percentage', "52 Weeks' Moving Range", 'Opening Price', "Day's Volume (Nos.)", 'Adjusted Opening Price', "Day's Trade (Nos.)", "Yesterday's Closing Price", 'Market Capitalization (mn)', 'Authorized Capital (mn)', 'Debut Trading Date', 'Paid-up Capital (mn)', 'Type of Instrument', 'Face/par Value', 'Market Lot', 'Total No. of Outstanding Securities', 'Sector', 'Address', 'Contact Phone', 'Fax', 'E-mail', 'Web Address', 'Company Secretary Name', 'Cell No.', 'Telephone No.', 'E-mail')


# Storing the data in the sqlite3 database

In [18]:
import sqlite3

In [19]:
con = sqlite3.connect('Stock_exchange.db')
cur = con.cursor()

In [20]:
cur.execute('''CREATE TABLE IF NOT EXISTS Stock_data 
('Company_name', 'Trading_name','Last Trading Price', 'Closing Price', 'Last Update', "Day's Range", 'Change*', "Day's Value (mn)", "52 Weeks' Moving Range", 'Opening Price', 'Change Percentage', "Day's Volume (Nos.)", 'Adjusted Opening Price', "Day's Trade (Nos.)", "Yesterday's Closing Price", 'Market Capitalization (mn)', 'Authorized Capital (mn)', 'Debut Trading Date', 'Paid-up Capital (mn)', 'Type of Instrument', 'Face/par Value', 'Market Lot', 'Total No. of Outstanding Securities', 'Sector', 'Address', 'Contact Phone', 'Fax', 'E-mail', 'Web Address', 'Company Secretary Name', 'Cell No.', 'Telephone No.', 'Email')''')

In [21]:
print('Total columns are :', len(Columns) )

Total columns are : 33


In [22]:
cur.executemany('''INSERT INTO Stock_data VALUES
                (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', Company_data)

In [23]:
con.commit()

In [24]:
con.close()

In [25]:
#Converting into csv
# convert list of tuples to list of list
Company_data = [list(data) for data in Company_data]


In [27]:
import pandas as pd
df = pd.DataFrame(Company_data, columns=Columns)

In [29]:
#Converting into csv
df.to_csv('DSE .csv', index=False)